In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

#### Regional Data electricity 

In [2]:
df_ele = pd.read_csv( "/home/onyxia/work/Python_Project_2A/data/RTE/data_ele-2020-2024.csv", low_memory=False)  
df_ele.head()

,Périmètre,Nature,Date,Heures,Consommation
0,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:00,8911.0
1,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:30,8814.0
2,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:00,8589.0
3,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:30,8575.0
4,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,02:00,8560.0


In [3]:
df_ele['Datetime'] = pd.to_datetime(
    df_ele['Date'].astype(str) + ' ' + df_ele['Heures'].astype(str),
    errors='coerce'
)

In [4]:
df_ele.head()

,Périmètre,Nature,Date,Heures,Consommation,Datetime
0,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:00,8911.0,2020-01-01 00:00:00
1,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,00:30,8814.0,2020-01-01 00:30:00
2,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:00,8589.0,2020-01-01 01:00:00
3,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,01:30,8575.0,2020-01-01 01:30:00
4,Auvergne-Rhône-Alpes,Données définitives,2020-01-01,02:00,8560.0,2020-01-01 02:00:00


In [5]:
df_ele.shape

(1052352, 6)

In [6]:
df_ele = df_ele.set_index('Datetime')
df_ele_daily = (
    df_ele.groupby("Périmètre", group_keys=False).resample('D')
           .agg({'Consommation': 'sum',
                  'Périmètre': 'first',
                 'Nature': 'first'})
           .reset_index()
)

In [7]:
df_ele_daily

,Datetime,Consommation,Périmètre,Nature
0,2020-01-01,403787.0,Auvergne-Rhône-Alpes,Données définitives
1,2020-01-02,443531.0,Auvergne-Rhône-Alpes,Données définitives
2,2020-01-03,434626.0,Auvergne-Rhône-Alpes,Données définitives
3,2020-01-04,395169.0,Auvergne-Rhône-Alpes,Données définitives
4,2020-01-05,400169.0,Auvergne-Rhône-Alpes,Données définitives
...,...,...,...,...
21919,2024-12-27,181429.0,Pays-de-la-Loire,Données consolidées
21920,2024-12-28,171307.0,Pays-de-la-Loire,Données consolidées
21921,2024-12-29,169980.0,Pays-de-la-Loire,Données consolidées
21922,2024-12-30,185431.0,Pays-de-la-Loire,Données consolidées


In [8]:
df_ele_daily['Périmètre'].count

<bound method Series.count of 0        Auvergne-Rhône-Alpes
1        Auvergne-Rhône-Alpes
2        Auvergne-Rhône-Alpes
3        Auvergne-Rhône-Alpes
4        Auvergne-Rhône-Alpes
                 ...         
21919        Pays-de-la-Loire
21920        Pays-de-la-Loire
21921        Pays-de-la-Loire
21922        Pays-de-la-Loire
21923        Pays-de-la-Loire
Name: Périmètre, Length: 21924, dtype: object>

In [9]:
df_ele_daily.columns

Index(['Datetime', 'Consommation', 'Périmètre', 'Nature'], dtype='object')

### Data Weather


In [10]:
df_weather = pd.read_csv("/home/onyxia/work/Python_Project_2A/data/weather/Region_France_weather_daily_2020_2024.csv")
df_weather.head(2)

,Datetime,year,month,day,Regions,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,GWETROOT,GWETTOP,PRECTOTCORR,PS,QV2M,RH2M,T2M,T2M_MAX,T2M_MIN,T2M_RANGE,WD10M,WS10M,WS2M
0,2020-01-01,2020,1,1,Auvergne-Rhône-Alpes,5.494,6.860,0.768,0.778,0.062,97.706,3.690,87.752,0.940,4.552,-0.758,5.310,291.10,1.938,1.168
1,2020-01-02,2020,1,2,Auvergne-Rhône-Alpes,4.532,6.448,0.768,0.778,0.042,97.484,4.194,87.350,2.642,7.794,-0.600,8.394,175.38,3.454,2.140


In [11]:
df_weather.rename(columns={"date":"Datetime","department":"Regions"}, inplace=True)
df_ele_daily.rename(columns={"Périmètre":"Regions"},inplace= True)


In [12]:
df_weather.columns

Index(['Datetime', 'year', 'month', 'day', 'Regions', 'ALLSKY_SFC_SW_DWN',
       'CLRSKY_SFC_SW_DWN', 'GWETROOT', 'GWETTOP', 'PRECTOTCORR', 'PS', 'QV2M',
       'RH2M', 'T2M', 'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'WD10M', 'WS10M',
       'WS2M'],
      dtype='object')

In [13]:
df_ele_daily.columns

Index(['Datetime', 'Consommation', 'Regions', 'Nature'], dtype='object')

### Joining data weather & data electricity by using regions and datetime

In [14]:
# Make sure both are datetime
df_weather['Datetime'] = pd.to_datetime(df_weather['Datetime'])
df_ele_daily['Datetime'] = pd.to_datetime(df_ele_daily['Datetime'])

# Merge on Datetime and Regions
df_final = pd.merge(
    df_ele_daily,
    df_weather,
    on=['Datetime', 'Regions'],
    how='inner'
)


In [15]:
df_final

,Datetime,Consommation,Regions,Nature,year,month,day,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,GWETROOT,...,PS,QV2M,RH2M,T2M,T2M_MAX,T2M_MIN,T2M_RANGE,WD10M,WS10M,WS2M
0,2020-01-01,403787.0,Auvergne-Rhône-Alpes,Données définitives,2020,1,1,5.494,6.860,0.768,...,97.706,3.690,87.752,0.940,4.552,-0.758,5.310,291.10,1.938,1.168
1,2020-01-02,443531.0,Auvergne-Rhône-Alpes,Données définitives,2020,1,2,4.532,6.448,0.768,...,97.484,4.194,87.350,2.642,7.794,-0.600,8.394,175.38,3.454,2.140
2,2020-01-03,434626.0,Auvergne-Rhône-Alpes,Données définitives,2020,1,3,3.846,6.454,0.766,...,97.456,4.872,89.086,4.570,8.598,1.944,6.654,196.00,2.976,1.748
3,2020-01-04,395169.0,Auvergne-Rhône-Alpes,Données définitives,2020,1,4,4.684,7.002,0.766,...,97.844,4.846,94.106,3.732,7.036,-0.244,7.280,276.04,3.874,2.512
4,2020-01-05,400169.0,Auvergne-Rhône-Alpes,Données définitives,2020,1,5,6.516,7.210,0.766,...,97.850,3.932,91.162,1.246,5.364,-1.542,6.906,236.76,3.654,2.338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21919,2024-12-27,181429.0,Pays-de-la-Loire,Données consolidées,2024,12,27,2.408,5.708,0.594,...,102.218,3.954,95.118,1.250,4.864,-2.760,7.624,116.98,2.014,1.236
21920,2024-12-28,171307.0,Pays-de-la-Loire,Données consolidées,2024,12,28,2.044,5.568,0.594,...,102.074,4.722,97.262,3.620,5.234,2.416,2.818,175.76,1.544,1.006
21921,2024-12-29,169980.0,Pays-de-la-Loire,Données consolidées,2024,12,29,2.210,5.526,0.594,...,102.238,4.312,94.026,2.890,5.036,1.216,3.820,137.36,1.374,0.898
21922,2024-12-30,185431.0,Pays-de-la-Loire,Données consolidées,2024,12,30,2.288,5.660,0.594,...,102.204,4.008,95.464,1.608,3.276,0.364,2.912,165.54,2.006,1.316


In [16]:
df_final.to_csv("temp_electricity.csv",index = False)